In [ ]:
!pip install accelerate transformers

In [ ]:
!pip install --upgrade langchain langchain-huggingface langchain-community

In [ ]:
!pip install langgraph

In [ ]:
!pip install wikipedia

In [ ]:
!pip install -U langchain langchain-community

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_core.tools import Tool
from langchain_core.prompts import PromptTemplate
from langgraph.prebuilt import create_react_agent
from datetime import datetime
import torch

In [7]:
model_id = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype=torch.float16,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [8]:
def segodnya():
    curdata = datetime.now()
    days = ["понедельник", "вторник", "среда", "четверг", "пятница", "суббота", "воскресенье"]
    months = ["января", "февраля", "марта", "апреля", "мая", "июня",
              "июля", "августа", "сентября", "октября", "ноября", "декабря"]
    return f"{curdata.day} {months[curdata.month-1]} {curdata.year} года, {curdata.hour:02d}:{curdata.minute:02d}:{curdata.second:02d} ({days[curdata.weekday()]})"

In [9]:
date_tool = Tool(
    name="Получить_текущую_дату",
    func=segodnya,
    description="""Используй для вопросов о дате, времени, дне недели."""
)

In [10]:
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.1,
    device_map="auto"
)


Device set to use cpu


In [11]:
llm = HuggingFacePipeline(pipeline=hf_pipeline)

Промпт

In [12]:
agent_prompt = PromptTemplate.from_template("""
Ты — помощник с доступом к инструментам.
Инструменты:
{tools}
Если вопрос о дате или времени — используй инструмент "Получить_текущую_дату".
Иначе отвечай сам.
Вопрос: {input}
""")

In [13]:
agent = create_react_agent(
    model=model,
    tools=[date_tool],
    prompt=agent_prompt
)

/tmp/ipython-input-2185031680.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [14]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator

In [15]:
class AgentState(TypedDict):
    messages: Annotated[List[str], operator.add]
    next: str


In [16]:
graph = StateGraph(AgentState)

In [17]:
def call_agent(state: AgentState):
    input_text = state.get("messages", [""])[-1] if state.get("messages") else ""

    # Если вопрос о дате - используем инструмент
    if any(word in input_text.lower() for word in ["дата", "время", "день", "час"]):
        from datetime import datetime
        curdata = datetime.now()
        response = f"Сегодня {curdata.day}.{curdata.month}.{curdata.year}, {curdata.hour}:{curdata.minute}"
    else:
        # Иначе модель отвечает сама
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=50)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {"messages": [response], "next": "end"}

In [18]:
graph.add_node("agent", call_agent)
graph.set_entry_point("agent")
graph.add_edge("agent", END)
app = graph.compile()

In [24]:
test_queries = [
    "Какая сегодня дата?",
    "Сколько сейчас времени?",
    "Какой сегодня день недели?",
    "Что такое Python?"
]
results = []
for query in test_queries:
    print(f"Вопрос: {query}")
    result = app.invoke({"messages": [query], "next": "agent"})

    if "messages" in result:
        response = result["messages"][-1] if result["messages"] else "Нет ответа"
    else:
        response = str(result)

    print(f"Ответ: {response[:100]}..." if len(response) > 100 else f"Ответ: {response}")

    # Сохраняем результат для записи в файл
    results.append((query, response))

with open('prompt.txt', 'w', encoding='utf-8') as f:
    f.write(agent_prompt.template)


with open('res.txt', 'w', encoding='utf-8') as f:
    f.write("Тестирование агента с инструментом даты\n\n")

with open('res.txt', 'w', encoding='utf-8') as f:
    f.write("Тестирование агента с инструментом даты\n\n")
    # Используем уже сохраненные результаты
    for query, response in results:
        f.write(f"Вопрос: {query}\n")
        f.write(f"Ответ: {response}\n")


Вопрос: Какая сегодня дата?
Ответ: Сегодня 12.12.2025, 10:20
Вопрос: Сколько сейчас времени?
Ответ: Сегодня 12.12.2025, 10:20
Вопрос: Какой сегодня день недели?
Ответ: Сегодня 12.12.2025, 10:20
Вопрос: Что такое Python?
Ответ: Что такое Python? Что может делать?

Python — это высокоуровневый язык программирования, который был...
